<a href="https://colab.research.google.com/github/A-Peoples/NFL-Box-Score-Python/blob/main/Box_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Packages**

In [ ]:
!pip install nfl_data_py
import seaborn as sns
import pandas as pd
import numpy as np
import nfl_data_py as nfl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 47.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 24.12.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2025.1.1 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.1 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.


**Importing play-by-play data and editing values**

In [ ]:
pbp = nfl.import_pbp_data([2024])

#changing every value to represent the amount of points you earn from it
pbp['touchdown']  = np.where(pbp['touchdown'] == 1., 6, 0)
pbp['safety']  = np.where(pbp['safety'] == 1., 2, 0)
pbp['two_point_conv_result']  = np.where(pbp['two_point_conv_result'] == "success", 2, 0)
pbp['extra_point_result']  = np.where(pbp['extra_point_result'] == "good", 1, 0)
pbp['field_goal_result']  = np.where(pbp['field_goal_result'] == "made", 3, 0)
pbp['pick_6'] = np.where((pbp['interception'] == 1) & (pbp['touchdown'] == 6), 6, 0)
pbp['scoop'] = np.where((pbp['fumble'] == 1) & (pbp['touchdown'] == 6), 6, 0)

2024 done.
Downcasting floats.


<ipython-input-3-ad3eabcaa13a>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pbp['pick_6'] = np.where((pbp['interception'] == 1) & (pbp['touchdown'] == 6), 6, 0)
<ipython-input-3-ad3eabcaa13a>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pbp['scoop'] = np.where((pbp['fumble'] == 1) & (pbp['touchdown'] == 6), 6, 0)


**Creating the single game stats and score for every game**

In [ ]:
game_ids = pbp["game_id"].unique()

game_box_score = pd.DataFrame()
for game_id in game_ids:
  # filtering the play-by-play data so that it only applies to one game
  box_score = pbp.loc[pbp.game_id == game_id]
  box_score_home = box_score.loc[box_score.posteam == box_score.home_team]
  box_score_away = box_score.loc[box_score.posteam == box_score.away_team]
  # creating the features for the home team

  box_score_home_score = box_score_home["touchdown"].sum() + box_score_home["field_goal_result"].sum() + box_score_home["extra_point_result"].sum() + box_score_home['two_point_conv_result'].sum() + box_score_away['pick_6'].sum() - box_score_home['pick_6'].sum() + box_score_away['scoop'].sum() - box_score_home["scoop"].sum()
  box_score_home_rush = box_score_home["rushing_yards"].sum()
  box_score_home_pass = box_score_home["passing_yards"].sum()
  box_score_home_fumble = box_score_home["fumble"].sum()
  box_score_home_sack = box_score_home["sack"].sum()
  box_score_home_interception = box_score_home["interception"].sum()
  box_score_home_team = box_score['home_team'].unique()[0]
  box_score_home_safety = box_score_home["safety"].sum() / 2
  box_score_home_pick_6 = box_score_home['pick_6'].sum() / 6
  box_score_home_tpr = box_score_home['two_point_conv_result'].sum()

  #creating the features for the away team
  box_score_away_score = box_score_away["touchdown"].sum() + box_score_away["field_goal_result"].sum() + box_score_away["extra_point_result"].sum() + box_score_away['two_point_conv_result'].sum() + box_score_home['pick_6'].sum() - box_score_away['pick_6'].sum() - box_score_away['scoop'].sum() + box_score_home["scoop"].sum()
  box_score_away_rush = box_score_away["rushing_yards"].sum()
  box_score_away_pass = box_score_away["passing_yards"].sum()
  box_score_away_fumble = box_score_away["fumble"].sum()
  box_score_away_sack = box_score_away["sack"].sum()
  box_score_away_interception = box_score_away["interception"].sum()
  box_score_away_team = box_score['away_team'].unique()[0]
  box_score_away_safety = box_score_away["safety"].sum() / 2
  box_score_away_pick_6 = box_score_home['pick_6'].sum()
  box_score_away_tpr = box_score_away['two_point_conv_result'].sum()


  # creating the dataset based on the home team and away team features
  normal_box_score = pd.DataFrame({
      'game_id': [game_id],
      "home_team": [box_score_home_team],
      "away_team": [box_score_away_team],
      "home_score": [box_score_home_score + box_score_away_safety],
      "away_score": [box_score_away_score + box_score_home_safety],
      "home_rush_yards": [box_score_home_rush],
      "away_rush_yards": [box_score_away_rush],
      "home_pass_yards": [box_score_home_pass],
      "away_pass_yards": [box_score_away_pass],
      "home_interception": [box_score_home_interception],
      "away_interception": [box_score_away_interception],
      'home_sack': [box_score_home_sack],
      'away_sack': [box_score_away_sack],
      "home_fumble": [box_score_home_fumble],
      "away_fumble": [box_score_away_fumble],
      "home_safety": [box_score_away_safety],
      "away_safety": [box_score_home_safety],
      'away_two_point': [box_score_away_tpr],
      'home_two_point': [box_score_home_tpr],
      'home_pick_6': [box_score_home_pick_6],
      'away_pick_6': [box_score_away_pick_6]
  })
  game_box_score = pd.concat([game_box_score, normal_box_score])



**Creating the "margin" column**

In [ ]:
game_box_score['margin'] = game_box_score['home_score']  - game_box_score['away_score']
final_team = pd.DataFrame()
teams = game_box_score['home_team'].unique()


**Creating the teams stats for the 2024 season and creating power rankings**

In [ ]:
group_team = pd.DataFrame()
for team in teams:
  #creating the variables
  single_team = game_box_score.loc[(game_box_score['home_team'] == team) | (game_box_score['away_team'] == team)].head(17)
  wins = 0
  home_wins = 0
  home_losses = 0
  away_wins = 0
  away_losses = 0
  losses = 0
  c_wins = 0
  c_losses = 0
  margin = 0
  total_games = 0
  points_scored = 0
  points_allowed = 0
  pass_yards = 0
  rush_yards = 0
  pass_allowed = 0
  rush_allowed = 0
  turnover = 0
  turnover_allowed = 0

  #for loop used to create the rows
  for index, row in single_team.iterrows():
    #home team features
    if row['home_team'] == team:
      if row.home_score > row.away_score:
        wins += 1
        home_wins += 1
        points_scored += row.home_score
        points_allowed += row.away_score
        margin += row.margin
      elif row.home_score < row.away_score:
        losses += 1
        home_losses += 1
        margin += row.margin
        points_scored += row.away_score
        points_allowed += row.home_score


    #away team features
    elif row['away_team'] == team:
      if row.away_score > row.home_score:
        wins += 1
        away_wins += 1
        points_scored += row.away_score
        points_allowed += row.home_score
        margin += row.margin
      elif row.home_score > row.away_score:
        losses +=1
        away_losses +=1
        points_scored += row.away_score
        points_allowed += row.home_score
        margin += row.margin
    pass_yards += row['home_pass_yards']
    rush_yards += row['home_rush_yards']
    turnover += (row['home_fumble'] + row['home_interception'])
    turnover_allowed += (row['away_fumble'] + row['away_interception'])
    pass_allowed += row['away_pass_yards']
    rush_allowed += row['away_rush_yards']
  team_single = pd.DataFrame({
      "team": [team],
      "wins": [wins],
      "home_wins": [home_wins],
      "away_wins": [away_wins],
      "losses": [losses],
      "home_losses": [home_losses],
      "away_losses": [away_losses],
      'ties': [0],
      "margin": [margin],
      "pass_yards": [pass_yards],
      "rush_yards": [rush_yards],
      "pass_allowed": [pass_allowed],
      "rush_allowed": [rush_allowed],
      "turnover": [turnover],
      'turnover_allowed': [turnover_allowed]

  })
  group_team = pd.concat([group_team, team_single])
#creating the power rankings

group_team['points'] = (((group_team.wins * 15) + (group_team.losses * -10)) - ((group_team.pass_allowed + group_team.rush_allowed) * 0.01) +  ((group_team.pass_yards + group_team.rush_yards) * 0.01) +
                        ((group_team.turnover * 1.5) - (group_team.turnover_allowed * 1.5)) + group_team.margin).round()
group_team = group_team.sort_values(by='points', ascending=False)

power_rankings = group_team[['team', "points"]]


#final_ratings
power_rankings

,team,points
0,BUF,264.0
0,MIN,253.0
0,KC,239.0
0,DET,231.0
0,GB,207.0
0,WAS,204.0
0,PHI,197.0
0,DEN,192.0
0,BAL,179.0
0,ARI,88.0
